In [97]:
import requests
from requests import Response
import re
import os

METRICS_USER = os.environ.get('METRICS_USER') or 'user1'
METRICS_PASSWORD = os.environ.get('METRICS_PASSWORD') or 'pw1'

API_URL = "http://127.0.0.1:8000"
auth = (METRICS_USER, METRICS_PASSWORD)

In [98]:
def get_request_counter(response : Response, endpoint : str):
    regex_pattern = r'(?<=http_requests_total{handler="' + endpoint + r'",method="GET",status="2xx"} )\d+(?=\.0)'
    match = re.search(regex_pattern, response.text)
    if(match is None): return 0
    return int(match.group())


In [99]:
response =requests.get(f"{API_URL}/metrics/", auth=auth)
assert response.status_code == 200, "Invalid status code"
print("Success: Response code 200")

Success: Response code 200


In [100]:
initial_metrics_requests = get_request_counter(response, "/metrics/")
print(f"First call: {initial_metrics_requests} calls to /metrics")

First call: 0 calls to /metrics


In [101]:
#second call after first run
response =requests.get(f"{API_URL}/metrics/", auth=auth)
assert response.status_code == 200, "Invalid response code"
print("Success: Response code 200")

Success: Response code 200


In [102]:
metrics_requests = get_request_counter(response, "/metrics/")
assert metrics_requests > initial_metrics_requests
print(f"Second call: {metrics_requests} calls to /metrics")

Second call: 2 calls to /metrics


In [103]:
initial_region_requests = get_request_counter(response, "/region/")
print(f"Success: region endpoint has {initial_region_requests} requests")

Success: region endpoint has 0 requests


In [104]:
#call region endpoint
_ = requests.get(f"{API_URL}/region/")
response = requests.get(f"{API_URL}/metrics/", auth=auth)
assert response.status_code == 200, "Invalid status code"

region_requests = get_request_counter(response, "/region/")
assert region_requests > initial_region_requests, "/region requests counter did not increment"
print(f"Success: region endpoint has {region_requests} requests")

Success: region endpoint has 2 requests


In [106]:
regex_pattern = r'(?<=amarillo_trips_number_total )[0-9]{1,}(?=\.0)'
match = re.search(regex_pattern, response.text)
assert match is not None, "No amarillo_trips_number_total metric"
print(f"Custom metric visible: {match.group()} trips")

Custom metric visible: 19 trips
